In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [2]:
trainSet = pickle.load(open('dataSetTrain.pkl', 'rb'))
testSet = pickle.load(open('dataSetTest.pkl','rb'))

ValueError: unsupported pickle protocol: 3

In [ ]:
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32 , shape=[None,40,434])
y_ = tf.placeholder(tf.float32 , shape=[None,9])
w = tf.Variable(tf.zeros([17360,9]),tf.float32)
b = tf.Variable(tf.zeros([9]),tf.float32)
sess.run(tf.global_variables_initializer())
p = tf.reshape(x,[-1,17360])
y = tf.nn.softmax(tf.matmul(p,w) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
y_genre = trainSet['Genre']
genre = pd.get_dummies(y_genre)
genMatrix = genre.as_matrix()

In [ ]:
for i in range(30):
    mfcc = trainSet[i * 50 :50 * (i+1)]['mfcc'].values
#     mfccRe = mfcc.copy()
#     mfccRe.resize([50,40,434])
    x_ = np.zeros([50,40,434])
    for j in range(50):
        tem = mfcc[j].copy()
        tem.resize(40,434)
        x_[j,:] = tem
    train_step.run(feed_dict={x: x_ , y_ : genMatrix[i * 50 :50 * (i+1)]})

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

x_ = np.zeros([testSet.shape[0],40,434])
mfcc = testSet['mfcc'].values
for i in range(testSet.shape[0]):
    tem = mfcc[i].copy()
    tem.resize(40,434)
    x_[i,:] = tem

testGen = testSet['Genre']
testGenCat = pd.get_dummies(testGen).values
print(testGenCat)
acc = accuracy.eval(feed_dict={x : x_, y_ :testGenCat })
print(acc)